# `ETraceOp`: 资格迹算子

资格迹算子（`ETraceOp`）是一个用于定义模型连接和突触交互的算子。它根据模型的输入和参数，计算突触后电流。

In [4]:
import jax
import jax.numpy as jnp

import brainscale
import brainstate

## 内置的资格迹算子

目前，brainscale提供了一系列内置的资格迹算子，包括：

- [`brainscale.MatMulOp`](../apis/generated/brainscale.MatMulOp.rst): 矩阵乘法算子。
- [`brainscale.ConvOp`](../apis/generated/brainscale.Conv2dOp.rst): 一般性的卷积算子。
- [`brainscale.SpMatMulOp`](../apis/generated/brainscale.SpMatMulOp.rst): 稀疏矩阵向量乘法算子。
- [`brainscale.LoraOp`](../apis/generated/brainscale.LoraOp.rst): Lora算子。

这些资格迹算子往往需要配合模型参数``brainscale.ETraceParam``一起使用。

### ``brainscale.MatMulOp`` 矩阵乘法算子

[`brainscale.MatMulOp`](../apis/generated/brainscale.MatMulOp.rst) 算子支持矩阵乘法操作，适用于全连接层等场景。它的输入是矩阵$x$和参数$w$，输出是它们的乘积$y$。

- 输入$x$是一个矩阵。
- 参数$w$是一个字典，涵盖了权重矩阵字段``weight``和偏置向量字段``bias``。这个算子可以用于实现全连接层的前向传播。
- 输出$y$是一个矩阵。

`brainscale.MatMulOp`支持如下操作：

1. $y = x @ \mathrm{param['weight']} + \mathrm{param['bias']}$

In [2]:
brainscale.ETraceParam(
    {
        'weight': brainstate.random.rand(4, 5),
        'bias': brainstate.random.rand(5)
     },
    brainscale.MatMulOp()
)

ETraceParam(
  value={
    'bias': ShapedArray(float32[5]),
    'weight': ShapedArray(float32[4,5])
  },
  gradient=<ETraceGrad.adaptive: 'adaptive'>,
  op=MatMulOp(is_diagonal=False),
  is_etrace=True
)

2. `mask`操作：$y = x @ (\mathrm{param['weight']} * \mathrm{mask}) + \mathrm{param['bias']}$


In [3]:
brainscale.ETraceParam(
    {
        'weight': brainstate.random.rand(4, 5),
        'bias': brainstate.random.rand(5)
     },
    brainscale.MatMulOp(
       weight_mask=brainstate.random.rand(4, 5) > 0.5
    )
)

ETraceParam(
  value={
    'bias': ShapedArray(float32[5]),
    'weight': ShapedArray(float32[4,5])
  },
  gradient=<ETraceGrad.adaptive: 'adaptive'>,
  op=MatMulOp(is_diagonal=False),
  is_etrace=True
)

3. 权重函数：$y = x @ f(\mathrm{param['weight']}) + \mathrm{param['bias']}$

In [5]:
brainscale.ETraceParam(
    {
        'weight': brainstate.random.rand(4, 5),
        'bias': brainstate.random.rand(5)
     },
    brainscale.MatMulOp(
       weight_fn=jnp.abs
    )
)

ETraceParam(
  value={
    'bias': ShapedArray(float32[5]),
    'weight': ShapedArray(float32[4,5])
  },
  gradient=<ETraceGrad.adaptive: 'adaptive'>,
  op=MatMulOp(is_diagonal=False),
  is_etrace=True
)

4. 权重函数+mask操作：$y = x @ f(\mathrm{param['weight']} * \mathrm{mask}) + \mathrm{param['bias']}$

In [6]:
brainscale.ETraceParam(
    {
        'weight': brainstate.random.rand(4, 5),
        'bias': brainstate.random.rand(5)
     },
    brainscale.MatMulOp(
       weight_fn=jnp.abs,
       weight_mask=brainstate.random.rand(4, 5) > 0.5
    )
)

ETraceParam(
  value={
    'bias': ShapedArray(float32[5]),
    'weight': ShapedArray(float32[4,5])
  },
  gradient=<ETraceGrad.adaptive: 'adaptive'>,
  op=MatMulOp(is_diagonal=False),
  is_etrace=True
)

### ``brainscale.ConvOp`` 卷积算子

[`brainscale.ConvOp`](../apis/generated/brainscale.Conv2dOp.rst) 算子支持一般性的卷积操作，适用于卷积神经网络（CNN）等场景。它的输入是特征图$x$和参数$w$，输出是卷积结果$y$。

- 输入$x$是一个矩阵。
- 参数$w$是一个字典，涵盖了权重矩阵字段``weight``和偏置向量字段``bias``。这个算子可以用于实现全连接层的前向传播。
- 输出$y$是一个矩阵。

`brainscale.ConvOp`支持1D、2D、3D卷积等多种形式的卷积操作。其维度信息由`xinfo`参数指定。`xinfo`是一个``jax.ShapeDtypeStruct``对象，包含了输入特征图的形状和数据类型。比如，

- 当 `xinfo=jax.ShapeDtypeStruct((32, 3), jnp.float32)` 时，表示输入是一个形状为 `(32, 3)` 的2维张量（通道数为3，长度均为32），此时卷积是1D卷积。
- 当 `xinfo=jax.ShapeDtypeStruct((32, 32, 3), jnp.float32)` 时，表示输入是一个形状为 `(32, 32, 3)` 的3维张量（通道数为3，高度和宽度均为32），此时卷积是2D卷积。
- 当 `xinfo=jax.ShapeDtypeStruct((32, 32, 32, 3), jnp.float32)` 时，表示输入是一个形状为 `(32, 32, 32, 3)` 的4维张量（通道数为3，高度和宽度均为32，深度为32），此时卷积是3D卷积。




对于每种维度的输入数据的卷积操作，`brainscale.ConvOp`支持如下格式：

1. $y = x \mathrm{[convolution]} \mathrm{param['weight']} + \mathrm{param['bias']}$

In [ ]:
# 以2D卷积为例
brainscale.ETraceParam(
    {
        'weight': brainstate.random.rand(3, 3),
        'bias': jnp.zeros(16)
    },
    brainscale.ConvOp(
        xinfo=jax.ShapeDtypeStruct((32, 3, 3), jnp.float32),
        window_strides=[1, 1],
        padding='SAME',
    )
)

2. `mask`操作：$y = x \mathrm{[convolution]}  (\mathrm{param['weight']} * \mathrm{mask}) + \mathrm{param['bias']}$


In [ ]:
# 以2D卷积为例
brainscale.ETraceParam(
    {
        'weight': brainstate.random.rand(3, 3),
        'bias': jnp.zeros(16)
    },
    brainscale.ConvOp(
        xinfo=jax.ShapeDtypeStruct((32, 3, 3), jnp.float32),
        window_strides=[1, 1],
        padding='SAME',
        weight_mask=brainstate.random.rand(3, 3) > 0.5
    )
)

3. 权重函数：$y = x \mathrm{[convolution]}  f(\mathrm{param['weight']}) + \mathrm{param['bias']}$

In [ ]:
# 以2D卷积为例
brainscale.ETraceParam(
    {
        'weight': brainstate.random.rand(3, 3),
        'bias': jnp.zeros(16)
    },
    brainscale.ConvOp(
        xinfo=jax.ShapeDtypeStruct((32, 3, 3), jnp.float32),
        window_strides=[1, 1],
        padding='SAME',
        weight_fn=jnp.abs
    )
)

4. 权重函数+mask操作：$y = x \mathrm{[convolution]}  f(\mathrm{param['weight']} * \mathrm{mask}) + \mathrm{param['bias']}$

In [ ]:
# 以2D卷积为例
brainscale.ETraceParam(
    {
        'weight': brainstate.random.rand(3, 3),
        'bias': jnp.zeros(16)
    },
    brainscale.ConvOp(
        xinfo=jax.ShapeDtypeStruct((32, 3, 3), jnp.float32),
        window_strides=[1, 1],
        padding='SAME',
        weight_mask=brainstate.random.rand(3, 3) > 0.5,
        weight_fn=jnp.abs,
    )
)

### `brainscale.SpMatMulOp` 稀疏矩阵向量乘法算子

[`brainscale.SpMatMulOp`](../apis/generated/brainscale.SpMatMulOp.rst) 算子支持稀疏矩阵向量乘法操作，适用于图神经网络（GNN）等场景。它的输入是特征图$x$和参数$w$，输出是稀疏矩阵乘法结果$y$


`brainscale.SpMatMulOp` 与 `brainscale.MatMulOp` 做类似的操作，用于


## 自定义资格迹算子

自定义资格迹算子需要继承`brainscale.ETraceOp`类，并实现以下方法：

- `xw_to_y()`: 计算突触后电流 $y = f(x, w)$。

![](../_static/etraceop-xw2y.png)


- `yw_to_w()`: 计算突触后维度向量如何影响权重 $w = g(y, w)$，特别会应用于**学习信号**作用在**资格迹**上 $\frac{\partial \mathcal{L}^{t}}{\partial \mathbf{h}^{t}} \boldsymbol{\epsilon}^{t}$。

![](../_static/etraceop-yw2w.png)